In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import io
from utils.utils import ensure_dir, gram_schmidt

In [3]:
import numpy as np
import torch
import dgl
import os.path as osp

In [4]:
def print_abs_mean_std(X):
    abs_X = torch.abs(X)
    print(abs_X.mean(), abs_X.std())

In [5]:
dataset = 'livejournal'
data_root = 'datasets/instance_' + dataset

In [6]:
hp = {
    'emb_dim': 64,
    'use_lightgcn_coe': False,
}

In [7]:
info = io.load_yaml(osp.join(data_root, 'info.yaml'))
num_nodes = info['num_nodes']
num_edges = info['num_edges']

E_src = io.load_pickle(osp.join(data_root, 'train_undi_csr_src_indices.pkl'))
E_dst = io.load_pickle(osp.join(data_root, 'train_undi_csr_indices.pkl'))
indptr = io.load_pickle(osp.join(data_root, 'train_undi_csr_indptr.pkl'))

if hp['use_lightgcn_coe']:
    all_degrees = indptr[1:] - indptr[:-1]
    d_src = all_degrees[E_src]
    d_dst = all_degrees[E_dst]
    edge_weights = torch.FloatTensor(1 / (d_src * d_dst)).sqrt()
    del all_degrees, d_src, d_dst
else:
    edge_weights = torch.ones(len(E_src))

del indptr

In [8]:
idx = torch.stack((torch.LongTensor(E_src), torch.LongTensor(E_dst)))
A = torch.sparse_coo_tensor(
    idx, edge_weights, (num_nodes, num_nodes)
)

In [9]:
U0 = torch.FloatTensor(size=(num_nodes, hp['emb_dim']))

In [10]:
hp['std'] = 1 / hp['emb_dim']
# hp['std'] = 1.0

In [56]:
torch.nn.init.normal_(U0, 0, hp['std'])

tensor([[-0.0055,  0.0198,  0.0026,  ...,  0.0182,  0.0166, -0.0118],
        [-0.0095, -0.0167,  0.0303,  ..., -0.0068, -0.0296, -0.0200],
        [-0.0169,  0.0257, -0.0029,  ..., -0.0186, -0.0044, -0.0043],
        ...,
        [ 0.0247, -0.0143,  0.0028,  ..., -0.0095,  0.0010, -0.0078],
        [ 0.0139,  0.0005, -0.0217,  ..., -0.0137,  0.0075, -0.0213],
        [-0.0250, -0.0013, -0.0162,  ..., -0.0347,  0.0148, -0.0303]])

In [57]:
hp['orthogonal'] = True

In [58]:
if hp['orthogonal']:
    print("using gram_schmidt to make columns orthogonal")
    U0 = gram_schmidt(U0.T).T

using gram_schmidt to make columns orthogonal


In [59]:
hp['layers'] = 3

In [60]:
Us = [U0]
print_abs_mean_std(U0)
# propagation
for i in range(hp['layers']):
    U = torch.sparse.mm(A, Us[i])
    print_abs_mean_std(U)
    Us.append(U)

tensor(0.0004) tensor(0.0003)
tensor(0.0011) tensor(0.0015)
tensor(0.0222) tensor(0.1332)
tensor(3.0295) tensor(49.2215)


In [61]:
hp['alpha'] = [1, 1, 1e-1, 1e-2]
# hp['alpha'] = [0, 0, 0, 1]

In [62]:
out_emb_table = torch.zeros(size=U0.shape)
alpha = hp['alpha']
for i, U in enumerate(Us):
    out_emb_table += alpha[i] * U

In [63]:
results_dir = 'alpha_1_1_1e-1_1e-2'
results_root = osp.join('model_outputs/gnn_' + dataset,
                        'randne', results_dir)
ensure_dir(results_root)
io.save_yaml(osp.join(results_root, 'config.yaml'), hp)
torch.save(out_emb_table, osp.join(results_root, 'out_emb_table.pt'))

In [64]:
print_abs_mean_std(out_emb_table)

tensor(0.0327) tensor(0.5047)
